In [1]:
import collections
import itertools  
import matplotlib.pyplot as plt
import networkx as nx
import os
import tellurium as te
import tesbml
import os, sys
cwd = os.getcwd()
print(cwd)

/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/SBMLLint/notebook


In [2]:
#parent_dir = os.path.abspath(os.path.join(cwd, os.pardir, 'SBMLLint'))
parent_dir = os.path.abspath(os.path.join(cwd, os.pardir))
print(parent_dir)

/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/SBMLLint


In [3]:
os.chdir(parent_dir)
print(os.getcwd())

/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/SBMLLint


In [4]:
#import os, sys, inspect
#import inspect
#cmd_folder = os.path.realpath(os.path.abspath(os.path.split(inspect.getfile( inspect.currentframe() ))[0]))
#if cmd_folder not in sys.path:
#     sys.path.insert(0, cmd_folder)

In [5]:
from SBMLLint.common import constants as cn
from SBMLLint.common.molecule import Molecule
from SBMLLint.common.reaction import Reaction
from SBMLLint.games import print_model as pm

In [6]:
os.getcwd()

'/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/SBMLLint'

In [7]:
print(cwd)

/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/SBMLLint/notebook


In [8]:
num = 35
format_num = format(num, '03d')
document = tesbml.readSBML(os.path.join(os.getcwd(), 'SBMLLint/games/data/curated_' + format_num + '.xml'))
model = document.getModel()
pm.print_model(model)

<Model vilarOscillator "Vilar2002_Oscillator">
Reaction1: A + R -> C;
Reaction2: A -> EmptySet;
Reaction3: C -> R;
Reaction4: R -> EmptySet;
Reaction5: A + DA -> DAp;
Reaction6: DAp -> A + DA;
Reaction7: DA -> DA + MA;
Reaction8: DAp -> DAp + MA;
Reaction9: MA -> EmptySet;
Reaction10: MA -> A + MA;
Reaction11: A + DR -> DRp;
Reaction12: DRp -> A + DR;
Reaction13: DR -> DR + MR;
Reaction14: DRp -> DRp + MR;
Reaction15: MR -> EmptySet;
Reaction16: MR -> MR + R;


In [9]:
print(Molecule)
print(Reaction)

<class 'SBMLLint.common.molecule.Molecule'>
<class 'SBMLLint.common.reaction.Reaction'>


In [10]:
reaction1 = model.getReaction(0)
reaction2 = model.getReaction(2)

In [11]:
a = Reaction(reaction1)
b = Reaction(reaction2)

In [12]:
Reaction.reactions

[A + R -> C, C -> R]

In [14]:
print(Reaction.reactions[1].reactants)
print(Reaction.reactions[1].products)

[MoleculeStoichiometry(molecule=C, stoichiometry=1.0)]
[MoleculeStoichiometry(molecule=R, stoichiometry=1.0)]


In [15]:
a.products

[MoleculeStoichiometry(molecule=C, stoichiometry=1.0)]

In [16]:
b.reactants

[MoleculeStoichiometry(molecule=C, stoichiometry=1.0)]

In [17]:
b.products

[MoleculeStoichiometry(molecule=R, stoichiometry=1.0)]

In [18]:
a.category

'reaction_n_1'

In [19]:
b.category

'reaction_1_1'

In [20]:
a.identifier

'A + R -> C'

In [21]:
b.identifier

'C -> R'

In [22]:
type(Reaction.reactions[0])

SBMLLint.common.reaction.Reaction

In [63]:
#ReactionCategory = collections.namedtuple('ReactionCategory',
#    'category predicate')
MoleculeStoichiometry = collections.namedtuple('Molecule', 'molecule stoichiometry')

In [64]:
MoleculeStoichiometry(molecule = Molecule.molecules[0], stoichiometry = 2)

Molecule(molecule=a, stoichiometry=2)

In [24]:
"""Set of Molecules(SOM)"""

from SBMLLint.common import constants as cn
from SBMLLint.common.molecule import Molecule
from SBMLLint.common.reaction import Reaction
from SBMLLint.common.simple_sbml import SimpleSBML

import numpy as np

In [67]:
BRACKET_OPEN = "{"
BRACKET_CLOSE = "}"

class SOM(object):
    soms = []  # All SOMs. 
    def __init__(self, molecule):
        """
        :param molecules Molecule instances:
        :param reactions Reaction instances:
        """
        self.molecules = {molecule}
        self.reactions = set()
        self.identifier = self.makeId()
        self.__class__.addSOM(self)

    def __repr__(self):
        return self.identifier        
        
    def makeId(self):
        """
        Creates an identifier for the SOM to uniquely
        identifies its elements.
        :return str:
        """
        def joinMoleculeNames(molecules):
          names = [m.name for m in molecules]
          names.sort()
          return ' + '.join(names)
        #
        identifier = "%s%s%s" % (
            BRACKET_OPEN, 
            joinMoleculeNames(list(self.molecules)),
            BRACKET_CLOSE
            )
        return identifier
        
    @classmethod    
    def addSOM(cls, new_som):
        if any([new_som.molecules.intersection(s.molecules) for s in cls.soms]):
          pass
        else:
          cls.soms.append(new_som)
        

In [68]:
SOM(Molecule.molecules[0])
SOM(Molecule.molecules[1])

{R}

In [72]:
SOM.soms[0].identifier

'{A}'

NameError: name 'string' is not defined

In [39]:
soms = [{'a', 'b'}, {'c', 'd'}]
new_som1 = {'a'}
new_som2 = {'e'}
print(any([new_som1.intersection(s) for s in soms]))
print(any([new_som2.intersection(s) for s in soms]))

True
False


In [33]:
bool(set())

False

In [ ]:
    def __init__(self, molecule):
        self.molecules = {molecule}
        self.reactions = set()
        SOM.all.append(self)

    def merge(self, som, reaction):
        self.molecules = self.molecules.union(som.molecules)
        self.reactions = self.reactions.union(som.reactions)
        self.reactions.add(reaction)
        return self

    @classmethod
    def initialize(som, molecule):
        return som(molecule)

    @classmethod
    def find(som, molecule):
        for som in SOM.all:
            if molecule in som.molecules:
                return som